In [1]:
# %matplotlib widget
%matplotlib inline

import edrixs
import numpy as np
import scipy as sp

import edrixs_utils
from edrixs_utils import rmat_t2o_DFT, prepare_slater, prepare_hopping

# Number of electrons

In [2]:
nd = 3
norb_d = 10
norb_bath = 10
nbath = 1
v_noccu  = nd + nbath*norb_d
shell_name = ('d', 'p') # valence and core shells for calculations

# Load atomic data from edrixs database

In [3]:
info  = edrixs.utils.get_atom_data('Cr', '3d', nd, edge='L3')
zeta_d_i = info['v_soc_i'][0]
zeta_d_n = info['v_soc_n'][0]
c_soc = info['c_soc']

# Build matrices describing interactions

In [4]:
U_dp = 6.0
U_dd = 4.0

ten_dq = 0.61

Delta = 0.2

scale_dd = 0.65
scale_dp = 0.5

In [5]:
# prepare slater parameters
slater = prepare_slater(info=info, U_dd=U_dd, U_dp=U_dp, 
                        scale_dd_i=scale_dd, scale_dd_n=scale_dd, scale_dp=scale_dp)

In [6]:
# below is the hopping matrix from DFT
H_MO = np.array([[ 0.288,   0.002,   0.001,   0.0  ,  -0.003,  -2.008, -0.011,  -0.019,  -0.001,   0.03 ,],
                 [ 0.002,   0.0  ,   0.003,   0.003,   0.003,  -0.001, -1.272,  -0.001,   0.007,  -0.002,],
                 [ 0.001,   0.003,   0.0  ,  -0.003,   0.003,   0.006, -0.002,  -1.272,  -0.002,  -0.001,],
                 [ 0.0  ,   0.003,  -0.003,   0.288,   0.0  ,   0.001, -0.028,   0.024,  -2.008,   0.004,],
                 [-0.003,   0.003,   0.003,   0.0  ,   0.0  ,  -0.005, -0.001,  -0.002,  -0.004,  -1.272,],
                 [-2.008,  -0.001,   0.006,   0.001,  -0.005,  -0.796,  0.01 ,   0.015,   0.0  ,  -0.026,],
                 [-0.011,  -1.272,  -0.002,  -0.028,  -0.001,   0.01 , -2.277,  -0.017,   0.024,  -0.017,],
                 [-0.019,  -0.001,  -1.272,   0.024,  -0.002,   0.015, -0.017,  -2.277,  -0.021,  -0.017,],
                 [-0.001,   0.007,  -0.002,  -2.008,  -0.004,   0.0  ,  0.024,  -0.021,  -0.796,  -0.003,],
                 [ 0.03 ,  -0.002,  -0.001,   0.004,  -1.272,  -0.026, -0.017,  -0.017,  -0.003,  -2.277,],
                ])

# prepare general hopping matrix
hopping_i, hopping_n, E_p = prepare_hopping(H_MO=H_MO, ten_dq=ten_dq, Delta=Delta, nd=nd, U_dd=U_dd,
                                            U_dp=U_dp, zeta_d_i=zeta_d_i, zeta_d_n=zeta_d_n, printParams=False,
                                           )

In [7]:
# apply c-aixs magnetic field to simulate the magnetic order in CrI3
ext_B = np.array([0.00, 0.00, 0.005])
ext_B = np.dot(ext_B, rmat_t2o_DFT())
print(ext_B) # Vector of external magnetic field with respect to global `xyz`-axis.
on_which = 'spin'

[0.00288675 0.00288675 0.00288675]


In [8]:
trans_c2n = edrixs.tmat_c2r('d',True)
om_shift = 576.3
c_level = -om_shift - 5*E_p

# Calculate charge transfer energy

In [9]:
hopping_i_CT = np.zeros((20,20), dtype=complex)
hopping_n_CT = np.zeros((20,20), dtype=complex)

hopping_i_CT += hopping_i
hopping_n_CT += hopping_n



hopping_i_CT[0:10,10:20] = np.zeros((10,10), dtype=complex)
hopping_i_CT[10:20,0:10] = np.zeros((10,10), dtype=complex)
hopping_n_CT[0:10,10:20] = np.zeros((10,10), dtype=complex)
hopping_n_CT[10:20,0:10] = np.zeros((10,10), dtype=complex)

In [10]:
umat_chb = np.zeros((20, 20, 20, 20), dtype=complex)
umat_delectrons_chb = edrixs.get_umat_slater('d', *slater[0])
umat_chb[:norb_d, :norb_d, :norb_d, :norb_d] += umat_delectrons_chb

emat_rhb = np.zeros((20, 20), dtype='complex')
emat_rhb += hopping_i_CT
tmat = sp.linalg.block_diag(edrixs.tmat_r2c('d',True), edrixs.tmat_r2c('d',True))
emat_chb = edrixs.cb_op(emat_rhb, tmat)
v_orbl = 2
sx, sy, sz = edrixs.get_sx(v_orbl), edrixs.get_sy(v_orbl), edrixs.get_sz(v_orbl)
zeeman = ext_B[0] * (2 * sx) + ext_B[1] * (2 * sy) + ext_B[2] * (2 * sz)
emat_chb[0:norb_d, 0:norb_d] += zeeman

In [11]:
energies = []

for n_ligand_holes in [0, 1]:
    basis_d = edrixs.get_fock_bin_by_N(10, nd + n_ligand_holes)
    Hd = (edrixs.build_opers(2, emat_chb[:10, :10], basis_d)
          + edrixs.build_opers(4, umat_chb[:10, :10, :10, :10], basis_d))
    ed = sp.linalg.eigh(Hd)[0][0]

    basis_L = edrixs.get_fock_bin_by_N(10, 10 - n_ligand_holes)
    HL = (edrixs.build_opers(2, emat_chb[10:, 10:], basis_L)
          + edrixs.build_opers(4, umat_chb[10:, 10:, 10:, 10:], basis_L))
    eL = sp.linalg.eigh(HL)[0][0]
    energies.append(ed + eL)

print(f"Charge transfer energy is {energies[1] - energies[0]:.3f} eV")

Charge transfer energy is -1.265 eV
